Sistema baseado na rede de distribuição de gás natural

1 Duto

equações :
$$
\frac{\partial}{\partial t}
\begin{bmatrix}
T \\
V \\
w
\end{bmatrix}
=
\begin{bmatrix}
-w & 0 & -T \frac{V^{*} P'_T}{C_V} \\
0 & -w & V \\
- V^{*} P'_T & -V^{*} P'_V & -w
\end{bmatrix}
\frac{\partial}{\partial x}
\begin{bmatrix}
T \\
V \\
w
\end{bmatrix}
+
\begin{bmatrix}
\frac{f w^2 |w|}{2 D C_V} + \frac{q}{C_V} \\
0 \\
\frac{f w |w|}{2 D}
\end{bmatrix}
$$
Retirado da Tese de doutorado em Engenharia Industrial de Rodrigo Lima Meira.

| **Variável**      | **Valor**           | **Unidade**     |
|-------------------|---------------------|-----------------|
| 𝜀                 | 1,5 × 10⁻⁶          | m               |
| κₛₒₗₒ             | 0,89                | W/(m·K)         |
| Tₛₒₗₒ             | 15                  | °C              |
| zₛₒₗₒ             | 2                   | m               |


In [1]:
import casadi as ca
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from libs.viscosity import *
from libs.eos_database import *
from libs.gc_eos_soave import *
from libs.composicaogas import *
from libs.duto import *
from scipy.integrate import solve_ivp
from numpy import sum
from libs.duto_casadi import *
from libs.compressor_class import *
from libs.compression import *
from libs.simulation_duto import *
import torch

In [2]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

Vpp = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4000, None, 1, 0, Aij, volumn_desviation, 'gas')
visc = viscosity(mixture, volumn_desviation)
D = 0.42
# Criar instância do duto com os parâmetros necessários  # ajuste conforme seu import real

# Número de nós (comprimento de self.l)

comp = CompressorClass()
compressor = compression(gas, comp, visc)
meu_duto = duto_casadi(gas, visc, compressor=compressor, Lc=200000.0, D=0.42)

0.99365 [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]
[0.9896844965531123, 0.006138982539123434, 0.0015095858702762542, 0.0003019171740552508, 0.0003019171740552508, 0.0005535148191012933, 0.0004025562320736678, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003019171740552508, 0.0, 0.0008051124641473356, 0.0]


In [3]:
from scipy.optimize import fsolve
import numpy as np

# --- Entradas conhecidas ---
u0 = 700
m_dot = 36
# --- Função residual (só para as variáveis livres) ---
def algebraic_residual(z_free, u, compressor):
    """
    Calcula o residual das equações algébricas do compressor
    com T2 e V2 fixos.
    """
    z_full = [z_free[0], z_free[1],
              z_free[2], z_free[3],
              z_free[4], z_free[5],
              z_free[6], z_free[7],
              z_free[8]]
    
    params = [u, m_dot/4, 4000, 300]  
    res = compressor.character_dae(z_full, params)
    return np.array(res).flatten()

Phi, eta, Mach, Gimp, G2, Gdif, PHI, G2s, k = compression.character(
    compressor, m=m_dot/4, N=u0, Gi_1=gas
)

z_guess = [Gimp.T, Gimp.V, Gdif.T, Gdif.V, G2s.T, G2s.V, G2.T, G2.V, gas.V]


# --- Resolve as equações algébricas ---
z_free_sol = fsolve(algebraic_residual, z_guess, args=(u0, compressor))

# Reconstrói o vetor completo com T2, V2 fixos
z_sol = [z_free_sol[0], z_free_sol[1],
         z_free_sol[2], z_free_sol[3],
         z_free_sol[4], z_free_sol[5],
         z_free_sol[6], z_free_sol[7],
         z_free_sol[8]]

print("Solução das variáveis algébricas:")
print(z_sol)

Solução das variáveis algébricas:
[np.float64(320.96133173352456), np.float64(0.48333044297385386), np.float64(320.35851600459756), np.float64(0.49511683401837436), np.float64(338.8953958002702), np.float64(0.35506519067469255), np.float64(344.38357043732435), np.float64(0.36277097658886776), np.float64(0.583790379032516)]


In [4]:
from scipy.optimize import fsolve

def solve_vq(gas, T2, v2, Tq):

    # Objeto gas_T2 é fixo (T2, v2)
    gas_T2 = gas.copy_change_conditions(T2, None, v2, 'gas')

    # Equação residual: P2 - Pq = 0
    def residual(vq):
        gas_Tq = gas.copy_change_conditions(Tq, None, vq, 'gas')
        return gas_T2.P - gas_Tq.P

    # chute inicial (pode ser qualquer valor próximo)
    vq_guess = v2

    vq_solution = fsolve(residual, vq_guess)[0]
    return vq_solution
vq = solve_vq(gas, 344.38357043732435, 0.36277097658886776, 300)
print("vq =", vq)

vq = 0.2977912609200901


In [5]:
A = np.pi * (D**2) / 4 
v2 = 0.36277097658886776
MM = gas.mixture.MM_m

rho = 1 / (v2 / MM)

w = m_dot / (A * rho)

T2 = 344.38357043732435
Tq = 300
T_af = 270


gas_T2 = gas.copy_change_conditions(T2, None, v2, 'gas')

mu_T2 = visc.evaluate_viscosity(T2, gas_T2.P)

Re_T2 = rho * w * D / mu_T2

kappa_T2 = coef_con_ter(gas_T2)

h_t_T2 = meu_duto.coef_cov_fluid(kappa_T2, mu_T2, Re_T2, gas_T2)


rho_Tq = 1 / (vq / MM)
w_Tq = m_dot / (A * rho_Tq)
gas_Tq = gas.copy_change_conditions(Tq, None, vq, 'gas')
mu_Tq = visc.evaluate_viscosity(Tq, gas_Tq.P )

Re_Tq = rho_Tq * w_Tq * D / mu_Tq

kappa_Tq = coef_con_ter(gas_Tq)

h_t_Tq = meu_duto.coef_cov_fluid(kappa_Tq, mu_Tq, Re_Tq, gas_Tq)

print("P2 (kPa):", gas_T2.P)
print("rho_Tq:", rho_Tq)
print("mu_Tq:", mu_Tq)
print("Re_Tq:", Re_Tq)

print(kappa_T2)
print(kappa_Tq)

print("rho_T2:", rho)
print("mu_T2:", mu_T2)
print("Re_T2:", Re_T2)

print("ht_q:", h_t_Tq)
print("ht_2:", h_t_T2)



P2 (kPa): 7462.754285545521
rho_Tq: 54.65756571380305
mu_Tq: 1.1630563817268598e-05
Re_Tq: 9383450.349854732
[[4.86233535]]
[[4.3067231]]
rho_T2: 44.86727567288956
mu_T2: 1.2215347440624084e-05
Re_T2: 8934237.740730286
ht_q: [[1.47455415]]
ht_2: [[1.3762858]]


In [ ]:
from scipy.optimize import fsolve
import numpy as np

# --- Entradas conhecidas ---
u0 = 600
m_dot = 36
m_dot_agua = 20
A = np.pi * (D**2) / 4  
T_af = 280
# --- Função residual (só para as variáveis livres) ---
def residual_z9_z10_z11(z_free, z0a8):
    """
    Resolve apenas as 3 equações algébricas finais:
        a10, a11, a12
    usando como entrada apenas:
        - z0..z8 (já resolvidas no primeiro fsolve)
        - z9 = z_free[0]
        - z10 = z_free[1]
        - z11 = z_free[2]
    """

    # variáveis conhecidas (das primeiras 9 eqs)
    T2 = z0a8[6]
    v2 = z0a8[7]

    # variáveis livres deste fsolve
    Tq = z_free[0]
    vq = z_free[1]
    Tw = z_free[2]

    # === Propriedades em T2 ===
    MM = gas.mixture.MM_m
    rho = 1 / (v2 / MM)
    w = m_dot / (A * rho)

    gas_T2 = gas.copy_change_conditions(T2, None, v2, 'gas')
    mu_T2 = visc.evaluate_viscosity(T2, gas_T2.P)
    Re_T2 = rho * w * D / mu_T2
    kappa_T2 = coef_con_ter(gas_T2)
    h_t_T2 = meu_duto.coef_cov_fluid(kappa_T2, mu_T2, Re_T2, gas_T2)
    print(h_t_T2)
    # === Propriedades em Tq ===
    gas_Tq = gas.copy_change_conditions(Tq, None, vq, 'gas')
    mu_Tq = visc.evaluate_viscosity(Tq, gas_Tq.P)
    Re_Tq = rho * w * D / mu_Tq
    kappa_Tq = coef_con_ter(gas_Tq)
    h_t_Tq = meu_duto.coef_cov_fluid(kappa_Tq, mu_Tq, Re_Tq, gas_Tq)
    print(h_t_Tq)
    # === Equações ===
    a10 = m_dot*(h_t_T2 - h_t_Tq) - 300*meu_duto.T_ML(T2, Tq, Tw, T_af)
    a11 = m_dot_agua*4184*(T_af - Tw) - 300*meu_duto.T_ML(T2, Tq, Tw, T_af)
    a12 = gas_T2.P - gas_Tq.P

    a10 = float(np.array(a10).flatten()[0])
    a11 = float(np.array(a11).flatten()[0])
    a12 = float(np.array(a12).flatten()[0])
    return np.array([a10, a11, a12])

guess = [290, .3, 320]  # Tq, vq, Tw iniciais

z9_10_11 = fsolve(residual_z9_z10_z11, guess, args=(z_sol,))
print(z9_10_11)

In [ ]:
gas_p = gas.copy_change_conditions(z_sol[6], None, z_sol[7], 'gas')

n_nodes = len(meu_duto.l)
A = np.pi * (D**2) / 4  
MM = gas_p.mixture.MM_m  
v_kg = z_sol[7] / MM
rho = 1 / v_kg
w_teste = m_dot / (A * rho)
y0 = np.array([z_sol[6], z_sol[7], w_teste])

x_avaliar = meu_duto.l

# --- Criação do integrador CVODES ---
x = SX.sym('x')
y = SX.sym('y', 3)  # [T, V, w]
dy = meu_duto.estacionario(x, y)
f = Function('f', [x, y], [dy])

dae = {'x': y, 'p': x, 'ode': f(x, y)}

# --- Integração incremental ao longo do duto ---
y_current = y0.reshape(-1, 1)
Y_sol = [y_current.flatten()]

for i in range(1, len(x_avaliar)):
    dx = x_avaliar[i] - x_avaliar[i - 1]
    F_seg = integrator('F_seg', 'cvodes', dae,
                       {'tf': dx})
    res = F_seg(x0=y_current, p=x_avaliar[i - 1])
    y_current = res['xf']
    Y_sol.append(y_current.full().flatten())

Y_sol = np.array(Y_sol)
T_sol_estc = Y_sol[:, 0]
V_sol_estc = Y_sol[:, 1]
w_sol_estc = Y_sol[:, 2]

# --- Cálculo da pressão ---
P_sol_estc = []
for i in range(len(T_sol_estc)):
    gas2 = gas.copy_change_conditions(T_sol_estc[i], None, V_sol_estc[i], 'gas')
    P_current = gas2.P.item()
    P_sol_estc.append(P_current)

# print(f"{'x [m]':>8} | {'T [K]':>8} | {'V [m³/kg]':>10} | {'w [m/s]':>10} | {'P [Pa]':>10}")
# print("-"*60)
# for xi, Ti, Vi, wi, Pi in zip(x_avaliar, T_sol_estc, V_sol_estc, w_sol_estc, P_sol_estc):
#     print(f"{xi:8.2f} | {Ti:8.2f} | {Vi:10.6f} | {wi:10.6f} | {Pi:10.2f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Temperatura (K) ---
plt.figure()
plt.plot(x_avaliar, T_sol_estc, marker='o', linestyle='-')
for xi, Ti in zip(x_avaliar, T_sol_estc):
    plt.annotate(f'{Ti:.2f} K', xy=(xi, Ti), xytext=(0, 5), textcoords='offset points', ha='center', fontsize=9)
plt.title('Distribuição de Temperatura ao longo do duto')
plt.xlabel('x [km]')
plt.ylabel('Temperatura [K]')
plt.grid(True)

# --- Volume específico (m³/kg) ---
plt.figure()
plt.plot(x_avaliar, V_sol_estc, marker='s', linestyle='-')
for xi, Vi in zip(x_avaliar, V_sol_estc):
    plt.annotate(f'{Vi:.5f}', xy=(xi, Vi), xytext=(0, 5), textcoords='offset points', ha='center', fontsize=9)
plt.title('Distribuição de Volume Específico ao longo do duto')
plt.xlabel('x [m]')
plt.ylabel('Volume específico [m³/kg]')
plt.grid(True)

# --- Velocidade (m/s) ---
plt.figure()
plt.plot(x_avaliar, w_sol_estc, marker='^', linestyle='-')
for xi, wi in zip(x_avaliar, w_sol_estc):
    plt.annotate(f'{wi:.2f} m/s', xy=(xi, wi), xytext=(0, 5), textcoords='offset points', ha='center', fontsize=9)
plt.title('Distribuição de Velocidade ao longo do duto')
plt.xlabel('x [m]')
plt.ylabel('Velocidade [m/s]')
plt.grid(True)


plt.figure()
plt.plot(x_avaliar, P_sol_estc, marker='^', linestyle='-')
for xi, Pi in zip(x_avaliar, P_sol_estc):
    plt.annotate(f'{Pi:.2f} kPa', xy=(xi, Pi), xytext=(0, 5), textcoords='offset points', ha='center', fontsize=9)
plt.title('Distribuição da Pressao ao longo do duto')
plt.xlabel('x [m]')
plt.ylabel('Pressao [kPa]')
plt.grid(True)

plt.show()


In [ ]:
meu_sistema = duto_casadi(gas, visc, compressor=compressor, Lc=200000.0, D=0.42)
# --- Condições iniciais ---
T0 = T_sol_estc
V0 = V_sol_estc
w0 = w_sol_estc
gas_temp = gas.copy_change_conditions(T_sol_estc[-1], None, V_sol_estc[-1], 'gas')
y0 = np.empty(n_nodes * 3)
y0[0::3] = T0
y0[1::3] = V0
y0[2::3] = w0
z0 = z_sol + [rho * A * w_sol_estc[0], gas_temp.P]
y0 = np.array(y0, dtype=float)
z0 = np.array(z0, dtype=float)
print(z0)
u0 = np.array([700.0, 30, 4000, 300.0, 2.1506, 290])
dt = 60
# --- Simulação temporal ---
n_steps = 1500 # número de passos
t_sim = np.linspace(0, n_steps * dt, n_steps)

y_sol = np.zeros((n_steps, len(y0)))

sim = SimuladorDuto(meu_sistema)
resultados = sim.run(y0, z0, u0, 4, 500, [700, 650, 750, 800], 60)
sim.plotar()

In [ ]:
RNN_train, x_train, y_train, x_min, x_max, y_min, y_max = sim.train_dataset(2)
print(RNN_train)